<a href="https://colab.research.google.com/github/andreitsev/colab_examples/blob/main/yandex_coderun_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id='home'></a>
### Contents

[172. Рассчитать pFound](#РассчитатьpFound)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os.path import join as pjoin
import sys
import time
import itertools
from tqdm.notebook import tqdm
import pickle
import json
import joblib
import collections

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
sns.set_style('whitegrid')

In [4]:
# Подключение данных гугл диска
from google.colab import drive
drive.mount('/content/drive')

gdrive_path = '/content/drive/MyDrive'

Mounted at /content/drive


In [ ]:
# #!cat /content/drive/MyDrive/spark_colab/spark_colab.sh
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# # install spark (change the version number if needed)
# !wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
# # unzip the spark file to the current folder
# !tar xf spark-3.0.0-bin-hadoop3.2.tgz
# !pip install -q findspark
# !pip install -q pyspark

In [48]:
%%time
!bash /content/drive/MyDrive/spark_colab/spark_colab.sh

file spark-3.0.0-bin-hadoop3.2.tgz already exists!
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
CPU times: user 1.18 s, sys: 162 ms, total: 1.34 s
Wall time: 2min 31s


In [49]:
# Подключаем спарк
#!bash /content/drive/MyDrive/spark_colab/spark_colab.sh
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['SPARK_HOME'] = "/content/drive/MyDrive/spark-3.0.0-bin-hadoop3.2"
# os.environ['SPARK_HOME'] = "/content/drive/MyDrive/ds_interview/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window as W
spark = (
    SparkSession
    .builder
    .getOrCreate()
)
spark

---

https://coderun.yandex.ru/

<a id='РассчитатьpFound'></a>
### 172. Рассчитать pFound
[back to contents](#home)

https://coderun.yandex.ru/problem/calculate-pfound?currentPage=1&pageSize=20&search=pFound

In [26]:
!pip install -q matplotlib-venn

<img width="734" alt="image" src="https://github.com/user-attachments/assets/bf2adf23-4afa-4d85-bba6-504b55492c9f">

https://disk.yandex.com/d/rOejo2Ec8mRGpA


In [89]:
problem_data_path = pjoin(gdrive_path, 'yandex_coderun_data', '172. Рассчитать pFound', 'hidden_task')
qid_url_rating = pd.read_csv(pjoin(problem_data_path, 'qid_url_rating.tsv'), sep='\t', header=None).rename(columns={0: 'id', 1: 'url', 2: 'relevance_score'})
qid_query = pd.read_csv(pjoin(problem_data_path, 'qid_query.tsv'), sep='\t', header=None).rename(columns={0: 'id', 1: 'query'})
hostid_url = pd.read_csv(pjoin(problem_data_path, 'hostid_url.tsv'), sep='\t', header=None).rename(columns={0: 'host_id', 1: 'url'})
print(f"qid_url_rating: {qid_url_rating.shape[0]:,} x {qid_url_rating.shape[1]:,}")
print(f"qid_query: {qid_query.shape[0]:,} x {qid_query.shape[1]:,}")
print(f"hostid_url: {hostid_url.shape[0]:,} x {hostid_url.shape[1]:,}")

qid_url_rating: 803 x 3
qid_query: 20 x 2
hostid_url: 803 x 2


In [90]:
from pyspark.sql import Row
spark.createDataFrame([Row(**row) for row in qid_url_rating.to_dict(orient='records')]).createOrReplaceTempView('qid_url_rating')
spark.createDataFrame([Row(**row) for row in qid_query.to_dict(orient='records')]).createOrReplaceTempView('qid_query')
spark.createDataFrame([Row(**row) for row in hostid_url.to_dict(orient='records')]).createOrReplaceTempView('hostid_url')

In [91]:
qid_url_rating.head()

,id,url,relevance_score
0,402111,http://24-job.com/board/job_australia/232-1-2-...,0.07
1,402111,http://24-job.com/board/job_australia/232-1-2-...,0.07
2,402111,http://802351.info/5964-v-avstralii.html,0.00
3,402111,http://auscommunity.com/blog/jobs/,0.00
4,402111,http://auscommunity.com/tag/%D1%84%D0%BE%D1%82...,0.00


In [92]:
qid_query.head()

,id,query
0,402111,работа фотографом в австралии
1,405851,производительность видеокарт
2,407522,ёлочные игрушки из пластиковых бутылок
3,409073,туту
4,438518,характеристика стрельцов


In [93]:
hostid_url.head()

,host_id,url
0,1,http://09spravki.ru/requisites.php
1,10,http://3pu.info/seo-tools/domains
2,1006,http://www.priroda.su/item/820
3,1007,http://www.prisnilos.su/znaki_zodiaka/strelets...
4,1009,http://www.prohotel.ru/hotel-108042/0/


In [104]:
for query in qid_query['query']:
  print(f'print("{query}")')

print("работа фотографом в австралии")
print("производительность видеокарт")
print("ёлочные игрушки из пластиковых бутылок")
print("туту")
print("характеристика стрельцов")
print("той терьер")
print("IL2 точность стрельбы")
print("home нет доменов")
print("средние века общая характеристика")
print("Ангел завтрашнего дня +текст")
print("что такое былина")
print("сергей зверев биография")
print("avz как пользоваться")
print("поделка букет цветов")
print("бесплатный просмотр камеди клаб")
print("Телефонный справочник Москвы")
print("тв-ведущие усть-илимск")
print("Гостиница Polaris, Сургут")
print("милая ты как солнце красивая")
print("КВН "Утомленные солнцем"")


In [94]:
id_host_id_df = spark.sql(
    """
    with joined_t as (
        select
          qid.id as id
          , qid.query as query
          , qid_url.url as url
          , qid_url.relevance_score as relevance_score
          , host_url.host_id as host_id
        from
          qid_query qid

          left join
            qid_url_rating qid_url
          on
            qid.id = qid_url.id

          left join
            hostid_url host_url
          on
            qid_url.url = host_url.url
      ),

      query_host_with_max_relevance as
      (
        select
          id
          , host_id
          , max(relevance_score) as max_revelance_score
        from
          joined_t
        group by
          id
          , host_id
      ),

      drop_duplicates as
      (
        select
          joined_t.*
        from
          joined_t
          left join
            query_host_with_max_relevance max_rel
          on
            1 = 1
            and joined_t.id = max_rel.id
            and joined_t.host_id = max_rel.host_id
        where
          1 = 1
          and joined_t.relevance_score = max_rel.max_revelance_score
      ),

      drop_duplicates_with_rn as
      (
        select
          *
          , row_number() over(partition by id order by relevance_score desc) as rn
        from
          drop_duplicates
        order by
          id
          , relevance_score desc
      ),

      table_for_pfound as
      (
        select
          *
        from
          drop_duplicates_with_rn
        where
          rn <= 10
      )

    select
      id
      , host_id
      , relevance_score
    from
      table_for_pfound
    """
).toPandas()
print(f"id_host_id_df: {id_host_id_df.shape[0]:,} x {id_host_id_df.shape[1]:,}")
id_host_id_df.head()

id_host_id_df: 200 x 3


,id,host_id,relevance_score
0,5308,160,0.14
1,5308,160,0.14
2,5308,208,0.14
3,5308,707,0.14
4,5308,1044,0.07


In [95]:
import typing as t

def pfound(
    documents: t.List[t.Tuple[int, int, float]],
    p_break: float=0.15,
) -> float:
  p_look = 1
  relevancies = [rel_score for _, _, rel_score in documents]
  pfound_ = 0
  for _, _, rel_score in documents:
    pfound_ += p_look*rel_score
    p_look = p_look*(1 - rel_score)*(1 - p_break)
  return pfound_

In [96]:
import itertools
data = [(id, host_id, rel) for id, host_id, rel in id_host_id_df.itertuples(index=False)]
data.sort(key=lambda x: (x[0], x[2]), reverse=True)

pfound_by_queries_dict = {}
for query_id, group in itertools.groupby(data, key=lambda x: x[0]):
  pfound_by_queries_dict[query_id] = pfound(documents=list(group))
print(len(pfound_by_queries_dict))

20


In [98]:
top_pfound_query_id, top_pfound_query_score = sorted(pfound_by_queries_dict.items(), key=lambda x: x[1], reverse=True)[0]
print(top_pfound_query_id, top_pfound_query_score)
qid_query.loc[(qid_query['id'] == top_pfound_query_id)]['query'].item()

409073 0.8136406842932786


'туту'